# AI-Hub SharePoint Agent

To create an AI agent that is capable to generate pages in SharePoint, you need to first connect to an LLM, in this case, OpenAI GPT-5 will be used

## Testing Connection to OpenAI API

OpenAI API needs an API-KEY to work. This is stored in an Azure vault-key

In [7]:
from openai import OpenAI
import re
import json
import sys
sys.path.append("..")

from agent.config import AgentConfig

In [8]:
agent_config = AgentConfig()

agent_config.load_secrets()

Connecting to the OpenAI API

*Code adapted, source [here](https://github.com/openai/openai-python)*

In [9]:
openai_client = OpenAI(api_key=agent_config.OPENAI_KEY.get_secret_value())

The main objective of the LLM is to generate a JSON containing the structure of a SharePoint page based on a specific topic, which will be a parameter

In [10]:
topic = "What is an LLM and how it works?"

In [11]:
response = openai_client.responses.create(
    model="gpt-4o",
    instructions="""
    You are a content generation agent specialized in designing modern SharePoint pages for public-facing communication. Your expertise lies in SharePoint layout design and in simplifying complex topics related to artificial intelligence for a general audience.

    Your role is to act as a science communicator and visual designer. You will generate the structure and content of a SharePoint page that introduces and explains the topic provided by the user.

    The SharePoint page must be:
    - Visually engaging
    - Easy to understand for non-technical readers
    - Structured for clarity and flow

    Each page must include:
    - A compelling and accessible title
    - Multiple sections with clear headers and explanatory content
    - Visual resources (e.g., images, diagrams, external links)
    - A final summary that reinforces key ideas
    - A link to a YouTube video that helps explain the topic further

    Output format: JSON with the following fields:
    - "title": string
    - "sections": array of objects with "header" and "content"
    - "visuals": array of strings (URLs or descriptions)
    - "summary": string

    You will receive a single input:  
    **{{topic}}** — the AI-related concept to be explained (e.g., Machine Learning, NLP, Neural Networks, AI Ethics, Azure AI Services, GCP Vertex AI, BigQuery ML).
        """,
    input = "Neural Networks",
    temperature=0.7

)

In [12]:
response.output_text

'```json\n{\n  "title": "Understanding Neural Networks: The Brains Behind AI",\n  "sections": [\n    {\n      "header": "Introduction to Neural Networks",\n      "content": "Neural networks are a class of algorithms inspired by the human brain, designed to recognize patterns and make decisions. They are a key component of many AI technologies, enabling machines to learn from data."\n    },\n    {\n      "header": "How Neural Networks Work",\n      "content": "A neural network consists of layers of nodes, or \'neurons.\' Each neuron receives input, processes it, and passes it to the next layer. The network learns by adjusting the connections between neurons based on the data provided."\n    },\n    {\n      "header": "Types of Neural Networks",\n      "content": "There are various types of neural networks, including feedforward, convolutional, and recurrent networks, each suited for different tasks such as image recognition, language processing, and time-series prediction."\n    },\n   

In [13]:
def parse_openai_json(text: str) -> dict:
    """
    Parses the text generated by OpenAI and returns a dict with the JSON.
    Removes code block delimiters and unnecessary spaces.

    Args:
        text (str): The text generated by OpenAI, possibly containing JSON inside code block delimiters.

    Returns:
        dict: The parsed JSON as a Python dictionary.
"""
    # Elimina delimitadores de bloque de código (```json ... ```)
    match = re.search(r"```json\s*(\{.*\})\s*```", text, re.DOTALL)
    if match:
        json_str = match.group(1)
    else:
        # Si no hay delimitadores, intenta buscar el primer objeto JSON
        match = re.search(r"(\{.*\})", text, re.DOTALL)
        if match:
            json_str = match.group(1)
        else:
            raise ValueError("No se encontró un bloque JSON válido en el texto.")

    return json.loads(json_str)

In [14]:
parsed_json = parse_openai_json(response.output_text)

In [15]:
parsed_json

{'title': 'Understanding Neural Networks: The Brains Behind AI',
 'sections': [{'header': 'Introduction to Neural Networks',
   'content': 'Neural networks are a class of algorithms inspired by the human brain, designed to recognize patterns and make decisions. They are a key component of many AI technologies, enabling machines to learn from data.'},
  {'header': 'How Neural Networks Work',
   'content': "A neural network consists of layers of nodes, or 'neurons.' Each neuron receives input, processes it, and passes it to the next layer. The network learns by adjusting the connections between neurons based on the data provided."},
  {'header': 'Types of Neural Networks',
   'content': 'There are various types of neural networks, including feedforward, convolutional, and recurrent networks, each suited for different tasks such as image recognition, language processing, and time-series prediction.'},
  {'header': 'Applications of Neural Networks',
   'content': "Neural networks are used 